In [1]:
start_date = '2021-08-31'

In [2]:
import pandas as pd
import requests
import json
from datetime import datetime
from datetime import date
import time
#config info
from configparser import ConfigParser
config = ConfigParser()
config.read('../config.ini')
etherscan_api = config.get('ETHERSCAN','etherscan_api')

#header for parsing
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
start_time = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp())

#print(start_time)
# Data from https://optimistic.etherscan.io/

In [3]:
# Contract creator ids
creator_ids = [
    '0xdead1cb30B3CA13Cd67D1d6f4e2790d12484Fdd8',
    '0x03863f6ad36f1fcd908517e3c56c6b3fd3f50752'
]

latest_block_str = 'https://api-optimistic.etherscan.io/api?module=proxy&action=eth_blockNumber&apikey=' + etherscan_api
starting_block_str = 'https://api-optimistic.etherscan.io/api?module=block&action=getblocknobytime&timestamp='\
                + str(start_time) + '&closest=before&apikey=' + etherscan_api
start_block = requests.get(starting_block_str,headers = headers).json()
latest_block = requests.get(latest_block_str,headers = headers).json()

start_block = int(start_block['result'])
latest_block = int(latest_block['result'],base=16)

chunk_size = 10000 #calc 10000 blocks at a time?

#block_chunk
#print(start_block)
#print(latest_block)
#print(latest_block-start_block)
#print((latest_block-start_block)/chunk_size)

In [4]:
# Get contracts
df = pd.DataFrame()
apid = []
for addr in creator_ids:
    api_str = 'https://api-optimistic.etherscan.io/api?module=account&action=txlist&address='\
                + addr + '&startblock=' + str(1) + '&endblock=' + str(latest_block) +'&sort=asc&apikey=' + etherscan_api
    
    r = requests.get(api_str, headers=headers)
    x = r.json()
    x = pd.DataFrame(x).reset_index()
    if len(x.index) < 10000: #if all txs captured append
        x = pd.json_normalize(x['result'])
        if not x.empty:
            y = x.loc[(x['to'] == '') & (x['contractAddress'] != '')] #Where transactions are being sent to NULL (means contract creation)
            apid.append(y)
    else:
        for i in range(1, latest_block, chunk_size*10): #broader search for contract creations which should be in the earlier chunk
            if ((i - 1)/(chunk_size*10)) % 5 == 0:
                time.sleep(1)
            api_str = 'https://api-optimistic.etherscan.io/api?module=account&action=txlist&address='\
                    + addr + '&startblock=' + str(i) + '&endblock=' + str(i+(chunk_size*10)-1) +'&sort=asc&apikey=' + etherscan_api
            r = requests.get(api_str, headers=headers)
            x = r.json()
            #print(x)
            if not x == '':
                x = pd.DataFrame(x).reset_index()
                x = pd.json_normalize(x['result'])
            if not x.empty:
                y = x.loc[(x['to'] == '') & (x['contractAddress'] != '')] #Where transactions are being sent to NULL (means contract creation)
                apid.append(y)
#print(apid)
df = pd.concat(apid).reset_index()
df = df.drop_duplicates() # just in cas
contract_hashes = df['contractAddress'].to_numpy() #convert to an array
#print(contract_hashes)

In [6]:
#Get txs
df = pd.DataFrame()
apid = []
for addr in contract_hashes:
    api_str = 'https://api-optimistic.etherscan.io/api?module=account&action=txlist&address='\
            + addr + '&startblock=' + str(1) + '&endblock=' + str(latest_block)\
            + '&sort=desc&apikey=' + etherscan_api
    r = requests.get(api_str, headers = headers)
    x = r.json()
    x = pd.DataFrame(x).reset_index()
    if len(x.index) < 10000: #if all txs captured append
        x = pd.json_normalize(x['result'])
        if not x.empty:
            x = x.loc[x['to'] == str.lower(addr)] #Where transactions are being sent to the addresses
            apid.append(x)
    else: #else go through chunks
        for i in range(start_block, latest_block, chunk_size):
            if ((i - start_block)/chunk_size) % 5 == 0:
                time.sleep(1)
            api_str = 'https://api-optimistic.etherscan.io/api?module=account&action=txlist&address='\
                + addr + '&startblock=' + str(i) + '&endblock=' + str(i+chunk_size-1)\
                + '&sort=desc&apikey=' + etherscan_api

            r = requests.get(api_str, headers=headers)
            x = r.json()
            if not x == '':
                x = pd.DataFrame(x).reset_index()
                x = pd.json_normalize(x['result'])
            if not x.empty:
                x = x.loc[x['to'] == str.lower(addr)] #Where transactions are being sent to the addresses
                apid.append(x)
        
    
df = pd.concat(apid)
#print(df)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [7]:
#Aggregate values
df_trunc = pd.DataFrame()
df_trunc['dt'] = pd.to_datetime(df['timeStamp'],unit='s').dt.floor('d') #time is in seconds - trunc to day
df_trunc['gas'] = pd.to_numeric(df['gas'])
df_trunc['gasPrice'] = pd.to_numeric(df['gasPrice'])
df_trunc['gasUsed'] = pd.to_numeric(df['gasUsed'])

#print(df_trunc.dtypes)
df_trunc=df_trunc.groupby('dt').agg({'dt':'count','gas':'sum', 'gasPrice':'mean','gasUsed':'sum'})\
            .rename(columns={"dt":"num_txs","gas": "total_gas", "gasPrice":"avg_gas_price", "gasUsed":"total_gas_used"})

In [8]:
#df_export = pd.DataFrame()
#df_export['dt'] = pd.to_datetime(df['timeStamp'],unit='s') #time is in seconds
#df_export['hash'] = df['hash']
#df_export['to'] = df['to']
#df_export['gas'] = df['gas']
#df_export['gasPrice'] = df['gasPrice']
#df_export['gasUsed'] = df['gasUsed']

df_export = df_trunc
df_export = df_export.drop_duplicates() # just in case
export_str = 'optimism_oracle_' + date.today().strftime("%Y_%m_%d") + '.csv'

df_export.to_csv(export_str,index=True)